<a href="https://colab.research.google.com/github/fatimasadiq/NLP_Module_Project_hatespeechdetection/blob/main/Negation_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize, RegexpParser
#from nltk.tokenize import sent_tokenize, word_tokenize


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
s="You are not bad"
s2=s.split()
s2

['You', 'are', 'not', 'bad']

In [ ]:
neg =[]

In [ ]:
tokenized_words=word_tokenize(s)
for words in tokenized_words:
  tag_words=pos_tag(tokenized_words)

neg.append(tag_words)



In [ ]:
neg

[[('You', 'PRP'), ('are', 'VBP'), ('not', 'RB'), ('bad', 'JJ')]]

In [ ]:
print(type(neg))

<class 'list'>


In [ ]:
from nltk.corpus import wordnet
for i in s2:
  for words in wordnet.synsets(i):
    #print(words.hyponyms())

SyntaxError: ignored

In [ ]:
from nltk.corpus import wordnet
synonyms=[]
for i in s2:
  for words in wordnet.synsets(i):
    for lemma in words.lemmas():
      synonyms.append(lemma.name())

synonyms


['are',
 'ar',
 'be',
 'be',
 'be',
 'exist',
 'be',
 'be',
 'equal',
 'be',
 'constitute',
 'represent',
 'make_up',
 'comprise',
 'be',
 'be',
 'follow',
 'embody',
 'be',
 'personify',
 'be',
 'be',
 'live',
 'be',
 'cost',
 'be',
 'not',
 'non',
 'bad',
 'badness',
 'bad',
 'bad',
 'big',
 'bad',
 'tough',
 'bad',
 'spoiled',
 'spoilt',
 'regretful',
 'sorry',
 'bad',
 'bad',
 'uncollectible',
 'bad',
 'bad',
 'bad',
 'risky',
 'high-risk',
 'speculative',
 'bad',
 'unfit',
 'unsound',
 'bad',
 'bad',
 'bad',
 'forged',
 'bad',
 'defective',
 'badly',
 'bad',
 'badly',
 'bad']

In [ ]:
from nltk.corpus import wordnet
antonyms=[]
for i in s2:
  for words in wordnet.synsets(i):
    for lemma in words.lemmas():
      if lemma.antonyms():
        antonyms.append(lemma.antonyms()[0].name())

antonyms

['differ', 'good', 'goodness', 'good', 'unregretful']

In [ ]:
import pandas as pd

#Data Loading 
df=pd.read_csv('/content/AskFm_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,Idk I'm not gay,0
2,This person is 100% fake so beware Isabella!,1
3,Oof lmao how is she fake? Who's this lmao shit...,1
4,If your loved one left n cut all ties w you sh...,0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/negex-master.zip

Archive:  gdrive/My Drive/negex-master.zip
d136b972b5d214959807ad5de1881a3225541fe7
   creating: negex-master/
   creating: negex-master/CherryConText/
   creating: negex-master/CherryConText/trunk/
  inflating: negex-master/CherryConText/trunk/__init__.py  
  inflating: negex-master/CherryConText/trunk/cherryConText-bootstrap2-posix.py  
  inflating: negex-master/CherryConText/trunk/cherryConText-bootstrap2-win.py  
  inflating: negex-master/CherryConText/trunk/cherryConText-bootstrap2.py  
  inflating: negex-master/CherryConText/trunk/createCherryConTextVE-posix.py  
  inflating: negex-master/CherryConText/trunk/createCherryConTextVE-win.py  
  inflating: negex-master/CherryConText/trunk/createCherryConTextVE.py  
  inflating: negex-master/CherryConText/trunk/djangoplugin.py  
  inflating: negex-master/CherryConText/trunk/httplogger.py  
  inflating: negex-master/CherryConText/trunk/installCherryConText-posix.py  
  inflating: negex-master/CherryConText/trunk/installCherryConText-pos

In [ ]:
import re

def sortRules (ruleList):
    """Return sorted list of rules.
    
    Rules should be in a tab-delimited format: 'rule\t\t[four letter negation tag]'
    Sorts list of rules descending based on length of the rule, 
    splits each rule into components, converts pattern to regular expression,
    and appends it to the end of the rule. """
    ruleList.sort(key = len, reverse = True)
    sortedList = []
    for rule in ruleList:
        s = rule.strip().split('\t')
        splitTrig = s[0].split()
        trig = r'\s+'.join(splitTrig)
        pattern = r'\b(' + trig + r')\b'
        s.append(re.compile(pattern, re.IGNORECASE))
        sortedList.append(s)
    return sortedList

class negTagger(object):
    '''Take a sentence and tag negation terms and negated phrases.
    
    Keyword arguments:
    sentence -- string to be tagged
    phrases  -- list of phrases to check for negation
    rules    -- list of negation trigger terms from the sortRules function
    negP     -- tag 'possible' terms as well (default = True)    '''
    def __init__(self, sentence = '', phrases = None, rules = None, 
                 negP = True):
        self.__sentence = sentence
        self.__phrases = phrases
        self.__rules = rules
        self.__negTaggedSentence = ''
        self.__scopesToReturn = []
        self.__negationFlag = None
        
        filler = '_'
        
        for rule in self.__rules:
            reformatRule = re.sub(r'\s+', filler, rule[0].strip())
            self.__sentence = rule[3].sub (' ' + rule[2].strip()
                                                          + reformatRule
                                                          + rule[2].strip() + ' ', self.__sentence)
        for phrase in self.__phrases:
            phrase = re.sub(r'([.^$*+?{\\|()[\]])', r'\\\1', phrase)
            splitPhrase = phrase.split()
            joiner = r'\W+'
            joinedPattern = r'\b' + joiner.join(splitPhrase) +  r'\b'
            reP = re.compile(joinedPattern, re.IGNORECASE)
            m = reP.search(self.__sentence)
            if m:
                self.__sentence = self.__sentence.replace(m.group(0), '[PHRASE]'
                                                          + re.sub(r'\s+', filler, m.group(0).strip())
                                                          + '[PHRASE]')
    
#        Exchanges the [PHRASE] ... [PHRASE] tags for [NEGATED] ... [NEGATED] 
#        based on PREN, POST rules and if negPoss is set to True then based on 
#        PREP and POSP, as well.
#        Because PRENEGATION [PREN} is checked first it takes precedent over
#        POSTNEGATION [POST]. Similarly POSTNEGATION [POST] takes precedent over
#        POSSIBLE PRENEGATION [PREP] and [PREP] takes precedent over POSSIBLE 
#        POSTNEGATION [POSP].
              
        overlapFlag = 0
        prenFlag = 0
        postFlag = 0
        prePossibleFlag = 0
        postPossibleFlag = 0
        
        sentenceTokens = self.__sentence.split()
        sentencePortion = ''
        aScopes = []
        sb = []
        #check for [PREN]
        for i in range(len(sentenceTokens)):
            if sentenceTokens[i][:6] == '[PREN]':
                prenFlag = 1
                overlapFlag = 0

            if sentenceTokens[i][:6] in ['[CONJ]', '[PSEU]', '[POST]', '[PREP]', '[POSP]']:
                overlapFlag = 1
            
            if i+1 < len(sentenceTokens):
                if sentenceTokens[i+1][:6] == '[PREN]':
                    overlapFlag = 1
                    if sentencePortion.strip():
                        aScopes.append(sentencePortion.strip())
                    sentencePortion = ''
            
            if prenFlag == 1 and overlapFlag == 0:
                sentenceTokens[i] = sentenceTokens[i].replace('[PHRASE]', '[NEGATED]')
                sentencePortion = sentencePortion + ' ' + sentenceTokens[i]
            
            sb.append(sentenceTokens[i])
        
        if sentencePortion.strip():
            aScopes.append(sentencePortion.strip())
        
        sentencePortion = ''
        sb.reverse()
        sentenceTokens = sb
        sb2 = []
        # Check for [POST]
        for i in range(len(sentenceTokens)):
            if sentenceTokens[i][:6] == '[POST]':
                postFlag = 1
                overlapFlag = 0

            if sentenceTokens[i][:6] in ['[CONJ]', '[PSEU]', '[PREN]', '[PREP]', '[POSP]']:
                overlapFlag = 1
            
            if i+1 < len(sentenceTokens):
                if sentenceTokens[i+1][:6] == '[POST]':
                    overlapFlag = 1
                    if sentencePortion.strip():
                        aScopes.append(sentencePortion.strip())
                    sentencePortion = ''
            
            if postFlag == 1 and overlapFlag == 0:
                sentenceTokens[i] = sentenceTokens[i].replace('[PHRASE]', '[NEGATED]')
                sentencePortion = sentenceTokens[i] + ' ' + sentencePortion
            
            sb2.insert(0, sentenceTokens[i])
        
        if sentencePortion.strip():
            aScopes.append(sentencePortion.strip())
        
        sentencePortion = ''
        self.__negTaggedSentence = ' '.join(sb2)
        
        if negP:
            sentenceTokens = sb2
            sb3 = []
            # Check for [PREP]
            for i in range(len(sentenceTokens)):
                if sentenceTokens[i][:6] == '[PREP]':
                    prePossibleFlag = 1
                    overlapFlag = 0

                if sentenceTokens[i][:6] in ['[CONJ]', '[PSEU]', '[POST]', '[PREN]', '[POSP]']:
                    overlapFlag = 1
            
                if i+1 < len(sentenceTokens):
                    if sentenceTokens[i+1][:6] == '[PREP]':
                        overlapFlag = 1
                        if sentencePortion.strip():
                            aScopes.append(sentencePortion.strip())
                        sentencePortion = ''
            
                if prePossibleFlag == 1 and overlapFlag == 0:
                    sentenceTokens[i] = sentenceTokens[i].replace('[PHRASE]', '[POSSIBLE]')
                    sentencePortion = sentencePortion + ' ' + sentenceTokens[i]
            
                sb3 = sb3 + ' ' + sentenceTokens[i]
        
            if sentencePortion.strip():
                aScopes.append(sentencePortion.strip())
            
            sentencePortion = ''
            sb3.reverse()
            sentenceTokens = sb3 
            sb4 = []
            # Check for [POSP]
            for i in range(len(sentenceTokens)):
                if sentenceTokens[i][:6] == '[POSP]':
                    postPossibleFlag = 1
                    overlapFlag = 0

                if sentenceTokens[i][:6] in ['[CONJ]', '[PSEU]', '[PREN]', '[PREP]', '[POST]']:
                    overlapFlag = 1
            
                if i+1 < len(sentenceTokens):
                    if sentenceTokens[i+1][:6] == '[POSP]':
                        overlapFlag = 1
                        if sentencePortion.strip():
                            aScopes.append(sentencePortion.strip())
                        sentencePortion = ''
            
                if postPossibleFlag == 1 and overlapFlag == 0:
                    sentenceTokens[i] = sentenceTokens[i].replace('[PHRASE]', '[POSSIBLE]')
                    sentencePortion = sentenceTokens[i] + ' ' + sentencePortion
            
                sb4.insert(0, sentenceTokens[i])
        
            if sentencePortion.strip():
                aScopes.append(sentencePortion.strip())
            
            self.__negTaggedSentence = ' '.join(sb4)
            
        if '[NEGATED]' in self.__negTaggedSentence:
            self.__negationFlag = 'negated'
        elif '[POSSIBLE]' in self.__negTaggedSentence:
            self.__negationFlag = 'possible'
        else:
            self.__negationFlag = 'affirmed'
        
        self.__negTaggedSentence = self.__negTaggedSentence.replace(filler, ' ')
        
        for line in aScopes:
            tokensToReturn = []
            thisLineTokens = line.split()
            for token in thisLineTokens:
                if token[:6] not in ['[PREN]', '[PREP]', '[POST]', '[POSP]']:
                    tokensToReturn.append(token)
            self.__scopesToReturn.append(' '.join(tokensToReturn))

    def getNegTaggedSentence(self):
        return self.__negTaggedSentence
    def getNegationFlag(self):
        return self.__negationFlag
    def getScopes(self):
        return self.__scopesToReturn
    
    def __str__(self):
        text = self.__negTaggedSentence
        text += '\t' + self.__negationFlag
        text += '\t' + '\t'.join(self.__scopesToReturn)

In [ ]:
from negex import *
import csv


def main():
    rfile = open(r'negex-master/negex.python/negex_triggers.txt')
    irules = sortRules(rfile.readlines())
    reports = csv.reader(open(r'negex-master/negex.python/Annotations-1-120.txt','rb'), delimiter = '\t')
    reports.next()
    reportNum = 0
    correctNum = 0
    ofile = open(r'negex_output.txt', 'w')
    output = []
    outputfile = csv.writer(ofile, delimiter = '\t')
    for report in reports:
        tagger = negTagger(sentence = report[2], phrases = [report[1]], rules = irules, negP=False)
        report.append(tagger.getNegTaggedSentence())
        report.append(tagger.getNegationFlag())
        report = report + tagger.getScopes()
        reportNum += 1
        if report[3].lower() == report[5]:
            correctNum +=1
        output.append(report)
    outputfile.writerow(['Percentage correct:', float(correctNum)/float(reportNum)])
    for row in output:
        if row:
            outputfile.writerow(row)
    ofile.close()

if __name__ == '__main__': main()

ModuleNotFoundError: ignored